In [1]:
import pandas as pd
import sklearn
# from sklearn import cross_validation
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
dataset = pd.read_excel(r'./200416_label0_dataset_sentence.xlsx')

In [2]:
tmp = []
for i in dataset['sender']:
    if i == 'customer':
        tmp.append(0)
    else:
        tmp.append(1)
dataset['sender_num'] = tmp
dataset.head()

,id,round,sender,texts,max_label,round_label,labels,sender_num
0,4227729258237820,1,customer,内涵 段子 联通 皮 点赞 中国联通 中国联通 客服 掌上 营业厅 内涵 段子 话题 封 郑...,3,2,1,0
1,4227729258237820,2,helpdesk,u,6,4,0,1
2,4227729258237820,3,customer,夸夸,3,0,1,0
3,4227729258237820,4,helpdesk,*,6,4,0,1
4,4121001149457180,1,customer,距离 反映 问题 已经 一个 星期 花粉 助手 D 荣耀 honor 荣耀 手机 华为 终端...,2,2,1,0


In [3]:
texts = list(dataset["texts"])
labels = list(dataset["labels"])
Round = list(dataset["round"])
sender_num = list(dataset["sender_num"])

In [5]:
# x_train=texts[1755:]
# x_test=texts[0:1755]
# y_train=labels[1755:]
# y_test=labels[0:1755]

In [4]:
import os
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

In [28]:
from sklearn.model_selection import cross_val_score
import numpy as np
def baseline_model(texts,labels):
    
    x_train = tfidf_vectorizer.fit_transform(texts).toarray()
#     x_test = tfidf_vectorizer.transform(x_test)
    svd = TruncatedSVD(n_components=10).fit(x_train)
    reduced = svd.transform(x_train)
    total = reduced.tolist()
    for item in range(len(total)):
        total[item] = np.append(total[item], sender_num[item])
    for item in range(len(total)):
        total[item] = np.append(total[item], Round[item])
    
    Models = ['Bernoulli NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree']
    function = [BernoulliNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
    perform_f1 = []
    perform_acc = []
    perform_recall = []
    perform_precision = []
    
    for i in tqdm(range(len(function))):
        model = function[i]
        performance = cross_val_score(model, total, labels, cv=10, scoring='accuracy')
        performance1 = cross_val_score(model, total, labels, cv=10, scoring='f1_weighted')
        performance2 = cross_val_score(model, total, labels, cv=10, scoring='recall')
        performance3 = cross_val_score(model, total, labels, cv=10, scoring='precision')
        
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")
        perform_acc.append(performance.mean())
        perform_f1.append(performance1.mean())
        perform_recall.append(performance2.mean())
        perform_precision.append(performance3.mean())

#         model.fit(x_train, y_train)
#         model.score(x_test, y_test)
#         e = y_test
#         p = model.predict(x_test)
#         print(metrics.classification_report(e,p))
#         perform_f1.append(metrics.f1_score(e,p,average='macro'))
#         perform_acc.append(metrics.accuracy_score(e,p))
        
    result_f1_table = pd.DataFrame({"Models":Models,"f1-scores":perform_f1,"accuracy":perform_acc
                                   ,"recall":perform_recall,"precision":perform_precision})
#     result_acc_table = pd.DataFrame({"Models":Models,"Result acc scores":perform_acc})
    return result_f1_table

In [29]:
import numpy as np
score1 = baseline_model(texts,labels)

====  BernoulliNB  ====
====  SVC  ====
====  LogisticRegression  ====
====  RandomForestClassifier  ====
====  KNeighborsClassifier  ====
====  DecisionTreeClassifier  ====



In [30]:
score1

,Models,f1-scores,accuracy,recall,precision
0,Bernoulli NB,0.802508,0.799593,0.920613,0.671578
1,Svm (linear),0.934208,0.935644,0.829955,0.999817
2,Logistic Regression,0.934208,0.935644,0.829955,0.999817
3,Random Forest,0.931852,0.931972,0.840747,0.976717
4,kNN,0.922059,0.922879,0.852771,0.937787
5,Decision Tree,0.905876,0.904983,0.881756,0.868630


In [13]:
# def baseline_model(x_train, x_test, y_train, y_test):
    
#     x_train = tfidf_vectorizer.fit_transform(x_train)
#     x_test = tfidf_vectorizer.transform(x_test)
    
#     Models = ['Bernoulli NB','Multinomial NB','Svm (linear)','Logistic Regression',
#               'Random Forest','kNN','Decision Tree']
#     function = [BernoulliNB(),MultinomialNB(),svm.SVC(kernel="linear"),LogisticRegression(),
#               RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier()]
#     perform_f1 = []
#     perform_acc = []
    
    
#     for i in tqdm(range(len(function))):
#         model = function[i]
# #         performance = cross_val_score(model, tfidf_vectorizer.fit_transform(texts), labels, cv=10, scoring'accuracy')
        
#         func = str(function[i])
#         print("==== ", func[0:func.index('(')], " ====")

#         model.fit(x_train, y_train)
#         model.score(x_test, y_test)
#         e = y_test
#         p = model.predict(x_test)
#         print(metrics.classification_report(e,p))
#         perform_f1.append(metrics.f1_score(e,p,average='macro'))
#         perform_acc.append(metrics.accuracy_score(e,p))
        
#     result_f1_table = pd.DataFrame({"Models":Models,"Result f1 scores":perform_f1})
#     result_acc_table = pd.DataFrame({"Models":Models,"Result acc scores":perform_acc})
#     return result_f1_table, result_acc_table

In [15]:
# score = baseline_model(x_train, x_test, y_train, y_test)

====  BernoulliNB  ====
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1624
           1       0.25      0.01      0.01       131

    accuracy                           0.92      1755
   macro avg       0.59      0.50      0.49      1755
weighted avg       0.88      0.92      0.89      1755

====  MultinomialNB  ====
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1624
           1       0.00      0.00      0.00       131

    accuracy                           0.93      1755
   macro avg       0.46      0.50      0.48      1755
weighted avg       0.86      0.93      0.89      1755

====  SVC  ====


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1624
           1       0.84      0.31      0.46       131

    accuracy                           0.94      1755
   macro avg       0.89      0.65      0.71      1755
weighted avg       0.94      0.94      0.93      1755

====  LogisticRegression  ====
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1624
           1       0.81      0.26      0.39       131

    accuracy                           0.94      1755
   macro avg       0.88      0.63      0.68      1755
weighted avg       0.93      0.94      0.93      1755

====  RandomForestClassifier  ====
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      1624
           1       0.70      0.24      0.36       131

    accuracy                           0.94      1755
   macro avg       0.82      0.62      0.66      1755
weighted

In [18]:
answer1=p.tolist()

In [19]:
answer1[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
import pandas as pd
dict1 = {"label": answer1,  
       }
select_df = pd.DataFrame(dict1)

In [21]:
select_df.to_csv('answer1.csv')